In [ ]:
!pip install fvcore

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pickle
import logging
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
from torchvision.transforms.functional import to_pil_image
import time
import psutil  # For CPU memory usage tracking
import torchvision.transforms as transforms
from datetime import datetime

In [ ]:

class Logger:
    @staticmethod
    def setup_logging(method_name):
        log_dir = f"/kaggle/working/logs/{method_name}"
        os.makedirs(log_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = f"{log_dir}/{method_name}_{timestamp}.log"
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        return log_dir

In [ ]:
def load_pretrained_model(model_name='i3d_r50', num_classes=10):
    model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    model.blocks[-1].proj = nn.Linear(model.blocks[-1].proj.in_features, num_classes)
    for param in model.blocks[-1].proj.parameters():
        param.requires_grad = True
    
    return model

In [ ]:
class VideoDataset(data.Dataset):
    def __init__(self, data, labels, transform=None):
        """
        Args:
            data (numpy array): Shape (N, T, H, W, C) - (samples, frames, height, width, channels)
            labels (numpy array): Corresponding labels
            transform (torchvision.transforms.Compose): Transformations for resizing & normalization
        """
        self.data = data
        self.labels = labels
        self.transform = transform

        # Define default transform if none is provided
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),  # Resize PIL Image
                transforms.ToTensor(),  # Convert back to Tensor
                transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
            ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Load a single video sample and process all frames.
        """
        video = torch.tensor(self.data[idx], dtype=torch.float32).permute(3, 0, 1, 2)  # (T, H, W, C) -> (C, T, H, W)

        # Convert each frame to PIL, apply transformations, and stack back
        resized_frames = []
        for frame in video.permute(1, 0, 2, 3):  # Convert (C, T, H, W) -> (T, C, H, W)
            pil_frame = to_pil_image(frame)  # Convert tensor to PIL Image
            resized_frame = self.transform(pil_frame)  # Resize and transform
            resized_frames.append(resized_frame)

        # Stack frames back into a tensor (C, T, H, W)
        video = torch.stack(resized_frames, dim=1)

        label = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure label is a tensor
        return video, label

In [ ]:
class ModelLoader:
    @staticmethod
    def load_pretrained_model(model_name='i3d_r50', num_classes=10):
        model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
        
        model.blocks[-1].proj = nn.Linear(model.blocks[-1].proj.in_features, num_classes)
        for param in model.blocks[-1].proj.parameters():
            param.requires_grad = True
        
        return model

In [ ]:
# model = ModelLoader.load_pretrained_model()

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, valid_loader, test_loader, device, method_name, fold, num_epochs=10, lr=0.001, writer=None):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', patience=3, factor=0.5)
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader
        self.num_epochs = num_epochs
        self.writer = writer
        self.method_name = method_name
        self.fold = fold
        self.best_val_acc = 0.0

    def train(self):
        start_time = time.time()  # Start timing
        process = psutil.Process()  # Get current process info
        max_memory_cpu = 0
        max_memory_gpu = 0

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss, correct, total = 0.0, 0, 0

            for inputs, labels in self.train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                # Track max memory usage
                max_memory_cpu = max(max_memory_cpu, process.memory_info().rss / (1024 * 1024))
                if torch.cuda.is_available():
                    max_memory_gpu = max(max_memory_gpu, torch.cuda.max_memory_allocated(self.device) / (1024 * 1024))

            train_loss = running_loss / total
            train_acc = correct / total

            val_acc, val_loss = self.evaluate(self.valid_loader, calculate_loss=True)
            self.scheduler.step(val_loss)

            epoch_time = time.time() - start_time
            logging.info(f"{self.method_name} - Fold {self.fold+1} - Epoch {epoch+1}/{self.num_epochs} - "
                         f"Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, "
                         f"Time: {epoch_time:.2f}s, CPU Mem: {max_memory_cpu:.2f}MB, GPU Mem: {max_memory_gpu:.2f}MB")

            if self.writer:
                self.writer.add_scalar(f'{self.method_name}/Loss/train/fold_{self.fold+1}', train_loss, epoch)
                self.writer.add_scalar(f'{self.method_name}/Loss/valid/fold_{self.fold+1}', val_loss, epoch)
                self.writer.add_scalar(f'{self.method_name}/Accuracy/train/fold_{self.fold+1}', train_acc, epoch)
                self.writer.add_scalar(f'{self.method_name}/Accuracy/valid/fold_{self.fold+1}', val_acc, epoch)
                self.writer.add_scalar(f'{self.method_name}/Memory/CPU/fold_{self.fold+1}', max_memory_cpu, epoch)
                if torch.cuda.is_available():
                    self.writer.add_scalar(f'{self.method_name}/Memory/GPU/fold_{self.fold+1}', max_memory_gpu, epoch)

    def evaluate(self, loader, calculate_loss=False):
        self.model.eval()
        correct, total, total_loss = 0, 0, 0.0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                
                all_preds.append(preds)
                all_labels.append(labels)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                if calculate_loss:
                    total_loss += self.criterion(outputs, labels).item() * inputs.size(0)
        
        accuracy = correct / total
        
        if calculate_loss:
            return accuracy, total_loss / total
        
        all_preds = torch.cat(all_preds).cpu().numpy()
        all_labels = torch.cat(all_labels).cpu().numpy()
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')
        conf_matrix = confusion_matrix(all_labels, all_preds)

        logging.info(f"{self.method_name} - Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
                     f"Recall: {recall:.4f}, F1-score: {f1:.4f}")
        logging.info(f"{self.method_name} - Confusion Matrix:\n{conf_matrix}")
        return accuracy, precision, recall, f1, conf_matrix


In [ ]:
class TrainingPipeline:
    def __init__(self, method, num_epochs=50, batch_size=8, lr=0.001):
        self.method = method
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr
        self.log_dir = self.setup_logging()
        self.writer = SummaryWriter(log_dir=f"/kaggle/working/tensorboard_logs/{method}")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logging.info(f"Using device: {self.device}")
        self.method_paths = {
            "combined": {"data": "/kaggle/input/threedpickles/classroom/classroom/data.pkl", "labels": "/kaggle/input/threedpickles/classroom/classroom/labels.pkl"},
            "student": {"data": "/kaggle/input/threedpickles/student/student/data.pkl", "labels": "/kaggle/input/threedpickles/student/student/labels.pkl"},
            "teacher": {"data": "/kaggle/input/threedpickles/teacher/teacher/data.pkl", "labels": "/kaggle/input/threedpickles/teacher/teacher/labels.pkl"}
        }
        self.data, self.labels = self.load_data()
        self.num_classes = len(set(self.labels))
    
    def setup_logging(self):
        log_dir = f"/kaggle/workgin/logs/{self.method}"
        os.makedirs(log_dir, exist_ok=True)
        return log_dir
    
    def load_data(self):
        logging.info(f"Loading {self.method} data...")
        with open(self.method_paths[self.method]["data"], 'rb') as f:
            data = pickle.load(f)
        with open(self.method_paths[self.method]["labels"], 'rb') as f:
            labels = pickle.load(f)
        logging.info(f"{self.method} dataset: {len(data)} samples, {len(set(labels))} classes")
        return data, labels
    def run(self):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        fold_results = []
    
        for fold, (train_idx, val_idx) in enumerate(skf.split(self.data, self.labels)):
            start_time = time.time()
            process = psutil.Process()
            torch.cuda.reset_max_memory_allocated()
    
            logging.info(f"Starting {self.method} - Fold {fold+1}/5")
            print(f"Starting {self.method} - Fold {fold+1}/5")
    
            x_train = [self.data[i] for i in train_idx]
            y_train = [self.labels[i] for i in train_idx]
            x_valid = [self.data[i] for i in val_idx]
            y_valid = [self.labels[i] for i in val_idx]
            x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, random_state=42)
    
            train_dataset = VideoDataset(x_train, y_train)
            valid_dataset = VideoDataset(x_valid, y_valid)
            test_dataset = VideoDataset(x_test, y_test)
    
            train_loader = data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
            valid_loader = data.DataLoader(valid_dataset, batch_size=self.batch_size, shuffle=False)
            test_loader = data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
    
            model = ModelLoader.load_pretrained_model(num_classes=self.num_classes)
    
            trainer = Trainer(
                model=model,
                train_loader=train_loader,
                valid_loader=valid_loader,
                test_loader=test_loader,
                device=self.device,
                method_name=self.method,
                fold=fold,
                num_epochs=self.num_epochs,
                lr=self.lr,
                writer=self.writer
            )
    
            trainer.train()
            test_accuracy, test_precision, test_recall, test_f1, conf_matrix = trainer.evaluate(test_loader)
    
            end_time = time.time()
            time_taken = end_time - start_time
            max_memory_cpu = process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB
            max_memory_gpu = torch.cuda.max_memory_allocated() / (1024 ** 2) if torch.cuda.is_available() else 0
    
            logging.info(f"Fold {fold+1} Time Taken: {time_taken:.2f} seconds")
            logging.info(f"Fold {fold+1} Peak CPU Memory Usage: {max_memory_cpu:.2f} MB")
            logging.info(f"Fold {fold+1} Peak GPU Memory Usage: {max_memory_gpu:.2f} MB")
    
            self.writer.add_scalar(f'{self.method}/Time/fold_{fold+1}', time_taken, fold)
            self.writer.add_scalar(f'{self.method}/Memory/CPU/fold_{fold+1}', max_memory_cpu, fold)
            self.writer.add_scalar(f'{self.method}/Memory/GPU/fold_{fold+1}', max_memory_gpu, fold)
    
            model_dir = f"/kaggle/working/model_weights/{self.method}"
            os.makedirs(model_dir, exist_ok=True)
            torch.save(model.state_dict(), f"{model_dir}/fold_{fold+1}_final.pth")
    
            fold_results.append({
                'fold': fold + 1,
                'accuracy': test_accuracy,
                'precision': test_precision,
                'recall': test_recall,
                'f1': test_f1,
                'time': time_taken,
                'cpu_memory': max_memory_cpu,
                'gpu_memory': max_memory_gpu
            })
    
        avg_accuracy = np.mean([res['accuracy'] for res in fold_results])
        avg_precision = np.mean([res['precision'] for res in fold_results])
        avg_recall = np.mean([res['recall'] for res in fold_results])
        avg_f1 = np.mean([res['f1'] for res in fold_results])
        avg_time = np.mean([res['time'] for res in fold_results])
        avg_cpu_memory = np.mean([res['cpu_memory'] for res in fold_results])
        avg_gpu_memory = np.mean([res['gpu_memory'] for res in fold_results])
    
        logging.info(f"{self.method} - Average results across folds:")
        logging.info(f"Accuracy: {avg_accuracy:.4f}")
        logging.info(f"Precision: {avg_precision:.4f}")
        logging.info(f"Recall: {avg_recall:.4f}")
        logging.info(f"F1-score: {avg_f1:.4f}")
        logging.info(f"Avg Time Taken: {avg_time:.2f} seconds")
        logging.info(f"Avg Peak CPU Memory: {avg_cpu_memory:.2f} MB")
        logging.info(f"Avg Peak GPU Memory: {avg_gpu_memory:.2f} MB")
    
        self.writer.add_hparams(
            {'method': self.method, 'epochs': self.num_epochs, 'batch_size': self.batch_size, 'lr': self.lr},
            {
                'hparam/accuracy': avg_accuracy,
                'hparam/precision': avg_precision,
                'hparam/recall': avg_recall,
                'hparam/f1': avg_f1,
                'hparam/time': avg_time,
                'hparam/cpu_memory': avg_cpu_memory,
                'hparam/gpu_memory': avg_gpu_memory
            }
        )
    
        self.writer.close()
        return fold_results


In [ ]:
def main():
    os.makedirs("/kaggle/working/logs", exist_ok=True)
    os.makedirs("/kaggle/working/model_weights", exist_ok=True)
    os.makedirs("/kaggle/working/tensorboard_logs", exist_ok=True)
    
    params = {'num_epochs': 100, 'batch_size': 32, 'lr': 0.001}
    methods = ["combined", "student", "teacher"]
    results = {}
    
    for method in methods:
        logging.info(f"==== Starting training for {method} dataset ====")
        pipeline = TrainingPipeline(method=method, **params)
        results[method] = pipeline.run()
        logging.info(f"==== Completed training for {method} dataset ====")
    
    logging.info("==== Summary of Results ====")
    for method in methods:
        avg_accuracy = np.mean([res['accuracy'] for res in results[method]])
        logging.info(f"{method}: Average Accuracy = {avg_accuracy:.4f}")

In [ ]:
main()